# Example QCoDeS Sweep Script for Dilution Refrigerator

## This block of code must be run- it imports all the relevant libraries and packages

In [13]:
import src
from src.sweep0d import Sweep0D
from src.sweep1d import Sweep1D
from src.sweep2d import Sweep2D
from src.daq_driver import Daq, DaqAOChannel, DaqAIChannel
from src.util import init_database
from src.tracking import *
import nidaqmx
import time
import numpy as np
import qcodes as qc
import os
from qcodes.dataset.measurements import Measurement
from qcodes import initialise_or_create_database_at, load_or_create_experiment, load_experiment_by_name
from qcodes.dataset.experiment_container import experiments
from qcodes.dataset.data_set import load_by_run_spec
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860

## The next blocks are specific to various instruments - you only need to run the blocks for the instruments you wish to control/get data from.

### If you want to use the Lakeshore temperature controller, run this to initialize it

In [ ]:
ls = Model_372('lakeshore_372', 'TCPIP::10.155.95.127::7777::SOCKET')
ls.ch01.enabled(True)
ls.ch02.enabled(True)
ls.ch03.enabled(True)
ls.ch05.enabled(True)
ls.ch06.enabled(True)
for ch in ls.channels:
    print(f'Temperature of {ch.short_name} ({"on" if ch.enabled() else "off"}): {ch.temperature()} {ch.units()}')

### Initialize the magnet here

In [ ]:
magnet = AMI430("Magnet", address="10.155.94.127", port=7180, has_current_rating=False)

### Initialize the DAQ here

In [2]:
daq = Daq("Dev1", "Daq")

## Optional: change your parameter labels to more descriptive values for clarity

In [ ]:
# Command takes the form
# <QCoDeS Parameter>.label="label descriptor"
daq.ai0.voltage.label="Isd"
daq.ai1.voltage.label="Vxx"
daq.ai2.voltage.label="Vxy"

## Create your sweep object here! 
### The arguments for the 1D sweep shown here are, in order, the parameter to be swept, the starting point, ending point, step value, delay between steps, flag to sweep back and forth, whether to save data, and plot data, and whether to plot the parameters against time or the sweeping parameter
### The only required arguments are the parameter, start, stop, and step values

In [3]:
sweep = Sweep1D(daq.ao0.voltage, 0, 5, 0.05, inter_delay = 0.1, bidirectional = True, save_data = True, 
            plot_data = True, x_axis_time = 1, plot_bin=1, continual=False)

## Tell the sweep which other parameters to track during the sweep. Here, we have added the magnetic field, the temperature, and a couple DAQ channels

In [4]:
sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai3.voltage)

## Now, we initialize/create the database we are saving to (if you want to save data)
## You can run this code between each run to give the data identifiers through *exp_name* and *sample_name*

In [5]:
try:
    # Make sure database_name and the path are set to the correct values!
    database_name = "temp_data.db"
    exp_name = "hardcoded"
    sample_name = "Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others"
    init_database(database_name, exp_name, sample_name, sweep)
except:
    print("Error opening database")

Error opening database


## Now, we can start the sweep! This will automatically start data collection, plotting, saving, etc

In [6]:
%matplotlib qt
sweep.start(ramp_to_start=False)

Sweeping ao0 Voltage to 5 V
Starting experimental run with id: 4. 
Done with the sweep, ao0 Voltage=-3.191891195797325e-16


In [ ]:
sweep.datasaver.add_result(('time', 1111111) (sweep.))

### To stop the sweep, run the stop command below. To restart it, you can run the start command from above again.

In [ ]:
sweep.start()

# EXPORT DATA TO CSV FILE

In [11]:
# Ensure that the names of the database/experiment/sample are right
filepath = os.environ['MeasureItHome'] + '\\Databases\\' + '.db'
initialise_or_create_database_at(filepath)
exp = load_experiment_by_name('Example experiment- Magnetic field dependence')
exp.data_sets()

ValueError: Many experiments matching your request found:
exp_id:1 (Example experiment- Magnetic field dependence-Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others) started at (1579823376.1693838)
exp_id:2 (Example experiment- Magnetic field dependence-Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others) started at (1579823611.8670948)
exp_id:3 (Example experiment- Magnetic field dependence-Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others) started at (1579824441.2748234)

In [71]:
#pick out the run id that you want-- the number after results, before the @
run_id = 4

In [79]:
dataset = load_by_run_spec(
    experiment_name=exp_name,
    sample_name=sample_name,
    captured_run_id=run_id
)

df = dataset.get_data_as_pandas_dataframe()
ds = pd.DataFrame()
for key,value in df.items():
    ds[[key]]=value[[key]]

In [74]:
# Choose where you want the CSV saved

ds.to_csv(os.environ['MeasureItHome'] + '\\Databases\\' + 'temp_data_from_pandas2.csv')

In [14]:
experiments()

[Example experiment- Magnetic field dependence#Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others#1@C:\Users\erunb\MeasureIt\Databases\temp_data.db
 ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------,
 Example experiment- Magnetic field dependence#Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others#2@C:\Users\erunb\MeasureIt\Databases\temp_data.db
 ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 1-results-1-Dev1_ai1_voltage,Dev1_ai0_voltage,Dev1_ao0_voltage,time,Dev1_ai3_voltage-1015,
 Example experiment- Magnetic field dependence#Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others#3@C:\Users\erunb\MeasureIt\Databases\temp_data.db
 -----------------------------------------

In [ ]:
%matplotlib qt
plot.create_figs()

In [ ]:
plot.clear()

In [ ]:
plot.create_figs()

In [ ]:
plot.isRunning()

In [ ]:

sweep.ramp_to(0.1)

In [ ]:
import matplotlib.pyplot as plt
plt.get_fignums()

In [ ]:
plt.figure(3)

In [ ]:
sweep.resume()

In [ ]:
sweep.runner.datasaver.dataset.get_data_as_pandas_dataframe()

In [ ]:
from src.* import *

In [ ]:
def foo(*args):
    for x in args:
        print(x[0])
        print(x[1])

In [ ]:
test = [('x',1), ('y',2), ('z',3)]

In [ ]:
foo(*test)

In [ ]:
__all__

In [ ]:
src.__all__

In [ ]:
Sweep1D()

In [ ]:
for mod in src.__all__:
    exec('from src.{} import *'.format(mod))

In [ ]:
Sweep1D()

In [ ]:
src.sweep1d.Sweep1D()

In [ ]:
Sweep0D()

In [ ]:
from src.sweep0d import *

In [ ]:
import sys
sys.modules[src.base_sweep]

In [ ]:
from src.sweep0d import Sweep0D

In [20]:
exps = experiments()
dsets = []
for exp in exps:
    for ds in exp.data_sets():
        new_ds = {}
        new_ds['run id'] = ds.run_id
        new_ds['exp name'] = ds.exp_name
        new_ds['sample name'] = ds.sample_name
        new_ds['db'] = ds.path_to_db
        ds = load_by_run_spec(
            experiment_name=new_ds['exp name'],
            sample_name=new_ds['sample name'],
            captured_run_id=new_ds['run id']
        )
        dsets.append(ds)

In [21]:
dsets

[results #1@C:\Users\erunb\MeasureIt\Databases\temp_data.db
 ----------------------------------------------------------
 Dev1_ao0_voltage - numeric
 Dev1_ai1_voltage - numeric
 Dev1_ai3_voltage - numeric
 time - numeric
 Dev1_ai0_voltage - numeric,
 results #2@C:\Users\erunb\MeasureIt\Databases\temp_data.db
 ----------------------------------------------------------
 Dev1_ao0_voltage - numeric
 Dev1_ai1_voltage - numeric
 Dev1_ai3_voltage - numeric
 time - numeric
 Dev1_ai0_voltage - numeric,
 results #3@C:\Users\erunb\MeasureIt\Databases\temp_data.db
 ----------------------------------------------------------
 Dev1_ao0_voltage - numeric
 Dev1_ai1_voltage - numeric
 Dev1_ai3_voltage - numeric
 time - numeric
 Dev1_ai0_voltage - numeric,
 results #4@C:\Users\erunb\MeasureIt\Databases\temp_data.db
 ----------------------------------------------------------
 Dev1_ao0_voltage - numeric
 Dev1_ai1_voltage - numeric
 Dev1_ai3_voltage - numeric
 time - numeric
 Dev1_ai0_voltage - numeric]

In [22]:
df=dsets[0].get_data_as_pandas_dataframe()

In [28]:
for name, ps in dsets[0].paramspecs.items():
    print(ps.label)

ao0 Voltage
ai1 Voltage
ai3 Voltage
Time
ai0 Voltage


In [40]:
import pandas as pd
export_ds = pd.DataFrame()
ds = dsets[0]
for key, value in df.items():
    #print(key)
    #print(value)
    label = find_param_label(key, ds)
    export_ds[[label]] = value[[key]] 
    
print(export_ds)
export_ds.to_csv(os.environ['MeasureItHome']+'\\Databases\\new_export.csv')

     ai0 Voltage (V)  ai1 Voltage (V)  ai3 Voltage (V)  ao0 Voltage (V)  \
0          -0.000594        -0.000101         0.698741     0.000000e+00   
1           0.049734         0.000064         0.663708     5.000000e-02   
2           0.099569        -0.000101         0.479335     1.000000e-01   
3           0.149569        -0.000265         0.315850     1.500000e-01   
4           0.199733        -0.000101         0.201377     2.000000e-01   
..               ...              ...              ...              ...   
198         0.149733        -0.000101         0.009110     1.500000e-01   
199         0.099569        -0.000265         0.010754     1.000000e-01   
200         0.049405        -0.000101         0.008616     5.000000e-02   
201        -0.000430         0.000064         0.010096    -3.191891e-16   
202        -0.000594        -0.000594         0.694794    -3.191891e-16   

     Time (s)  
0       4.141  
1       4.235  
2       4.344  
3       4.453  
4       4.547  
.. 

In [38]:
    def find_param_label(name, ds):
        use_name = name
        unit = None
        for p_name, ps in ds.paramspecs.items():
            if name == p_name:
                unit = ps.unit
                if ps.label is not None:
                    use_name = ps.label

        if unit is not None:
            use_name = f'{use_name} ({unit})'
        return use_name